#Análisis de datos

El análisis de datos es cada vez más empleado en las empresas. En esta guía abordaremos las siguientes acciones dentro del análisis de datos:
- Cargar los datos
- Preparar los datos
- Análisis descriptivo
- Análisis predictivo

Por:

Ferney Orlando Amaya Fernández


Inicialmente se cargan las librerías a emplear:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

#1.Cargar los datos

Se cargará un archivo en formato Excel desde un repositorio en Github:

In [ ]:
# Ruta del archivo en la Google Drive
ruta_archivo = 'https://github.com/FerneyOAmaya/Competencias-Digitales/blob/master/DB_AnalisisDatosRegresion.xlsx?raw=true'

#Cargar el archivo en formato Excel
df = pd.read_excel(ruta_archivo)

#2.Preparar los datos

La preparación de los datos consiste en realizar acciones que garanticen que los datos estén en condiciones óptimas para su análisis.

Se realizarán las siguientes acciones:
- Perfilado de datos
- Preprocesamiento o limpieza de datos

##2.1. Perfilado de los datos

Se ejecutan algunas instrucciones para comprender la estructura, el contenido y la calidad del conjunto de datos:

In [ ]:
print(df.info())

In [ ]:
# Conteo de valores nulos
print('-------------------------')
print('CONTEO DE VALORES NULOS')
print(df.isnull().sum())
print('-------------------------')
# Valores únicos en la columna 'Género'
print('VALORES ÚNICOS COLUMNA Género')
print(df['Género'].value_counts())

##2.2.Preprocesamiento o limpieza de datos

Incluye las siguientes acciones:
- Tratamiento de valores nulos
- Estandarización de los valores de la columna Género
- Manejo de valores atípicos

1. Se eliminan los registros que tengan valores nulos en el campo Evaluación Servicio:


In [ ]:
df = df.dropna(subset=['Evaluación Servicio','Producto'])

2. En la columna Edad se reemplazarán los valores nulos con el promedio de la columna:

In [ ]:
promedio_edad = df['Edad'].mean()
df['Edad'] = df['Edad'].fillna(promedio_edad)

3. Se estandarizan los valores de la columna Género, convirtiendo el texto en mayúsculas:

In [ ]:
df['Género'] = df['Género'].str.upper()

Se identifican visualmente los valores atípicos de la Edad con un gráfico boxplot

In [ ]:
df['Edad'].plot(kind='box')
plt.title('Boxplot de Ingreso')
plt.ylabel('Ingreso')
plt.show()

4. Para este ejemplo eliminaremos los valores atípicos para la Edad. Para hacerlo, eliminaremos los valores de edad superiores a 24 (ver Figura anterior):

In [ ]:
# Límites para outliers
limite_superior = 24
df = df[(df['Edad'] <= limite_superior)]

#3.Análisis descriptivo

El análisis descriptivo permite responder preguntas de lo que sucedió.

Se responderán las siguientes preguntas:

- ¿Quiénes compraron más, las mujeres o los hombres?
- ¿Cuáles fueron las ventas por año por Género?

1. Conteo por Género: Se presenta un diagrama circular del conteo por Género

In [ ]:
conteo_genero = df['Género'].value_counts()
conteo_genero.plot(kind='pie')  #Puede reemplazarse kind por bar
plt.title('Conteo por Género')
plt.show()

In [ ]:
conteo_genero

2. Ventas por año por género

Se presenta una Tabla de ventas (en millones) por año por género:

In [ ]:
ventas = df.groupby(['año', 'Género'])['Ventas'].sum().reset_index()
ventas

Se presenta un gráfico tipo scatter con las ventas (en millones) por año por género:

In [ ]:
ventas_mujer = ventas[ventas['Género'] == "MUJER"]
ventas_hombre = ventas[ventas['Género'] == "HOMBRE"]
plt.scatter(ventas_mujer['año'], ventas_mujer['Ventas'], label="Mujer")
plt.scatter(ventas_mujer['año'], ventas_hombre['Ventas'], label="Hombre")
plt.title('Ventas por Género y Año')
plt.xlabel('Año')
plt.ylabel('Ventas (en millones)')
plt.legend()
plt.xticks(ventas_hombre['año']) # Asegura que se muestren todos los años en el eje x
plt.grid(True)
plt.show()

#4.Análisis predictivo

El análisis predictivo permite responder preguntas de lo que ocurrirá en el futuro.

Se responderá la pregunta: Si se conserva la misma tendencia, ¿cuál será el valor de las ventas para los hombres hasta el año 2026?

Para predecir los valores futuros se empleará una interpolación:


In [ ]:
f_interp_hombre = interp1d(ventas_hombre['año'], ventas_hombre['Ventas'], kind='linear', fill_value='extrapolate')

Se calcula el valor de las ventas interpoladas hasta el año 2026:

In [ ]:
# Años para la interpolación
años_interp = np.arange(2014, 2027)

# Generar valor de Interpolación
interp_hombre = f_interp_hombre(años_interp)

Se visualizan los valores originales y los interpolados:

In [ ]:
plt.scatter(ventas_hombre['año'], ventas_hombre['Ventas']/1e6, label='Ventas Originales Hombre')
plt.scatter(ventas_mujer['año'], ventas_mujer['Ventas']/1e6, label='Ventas Originales Mujer')
plt.plot(años_interp, interp_hombre/1e6, color='blue', linestyle='--', label='Ventas Interpoladas Hombre')
plt.title('Ventas Totales por Año con Interpolación')
plt.xlabel('Año')
plt.ylabel('Ventas Totales (Millones)')
plt.legend()
plt.grid(True)
plt.show()